In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from PIL import Image
import time

In [2]:
import os
from scipy import ndimage, misc
import glob

images = []
i = 0   
labels = []

for root, dirnames, filenames in os.walk("proj3_images/Numerals/"):
    if dirnames!= []:
        dirrr = dirnames
    count = 0
    for filename in filenames:
        if ".png" in filename:
            count += 1
            filepath = os.path.join(root, filename)
            image = ndimage.imread(filepath, mode="L")
            image_resized = misc.imresize(image, (28, 28))
            images.append(image_resized)
    if count != 0:
        lMid = np.zeros((count,10))
        lMid[:,int(dirrr[i])] = 1
        if labels == []:
            labels = lMid
        else:
            labels = np.vstack((labels,lMid))
#         print(len(labels),len(labels[0]))
        i += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
uspsImages = np.asarray(images)
uspsLabels = np.asarray(labels)
uspsImages = uspsImages/255
uspsImages = 1 - uspsImages
uspsImages = uspsImages.reshape((-1,784))
meanUSPSImg = np.mean(uspsImages,0)[:,np.newaxis]
uspsImages = uspsImages.T
uspsLabels = uspsLabels.T
# print(meanUSPSImg.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
trainImages = mnist.train.images.T
testImages = mnist.test.images.T
trainLabels = mnist.train.labels.T
testLabels = mnist.test.labels.T
validationImages = mnist.validation.images.T
validationLabels = mnist.validation.labels.T
m = trainImages.shape[1]

batchSize = 100
printEvery = 2
nEpochs = 15
nClasses = 10
learningRate = 0.005 #IF used....
lambdaa = 0.05 #Regularization

In [5]:
x = tf.placeholder(tf.float32, [784,None])
t = tf.placeholder(tf.float32, [10,None])

In [6]:
def convolve2D(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

In [7]:
def maxpool2d(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1] ,strides=[1,2,2,1], padding='SAME')

In [8]:
def neuralNetwork(x):
    lambdaa = 0.1
    nH1 = 512
    weights = {'hiddenL1':tf.Variable(tf.random_normal([nH1,784])),
               'hiddenL2':tf.Variable(tf.random_normal([10,nH1]))}
    
    biases = {'hiddenL1':tf.Variable(tf.random_normal([nH1,1])),
              'hiddenL2':tf.Variable(tf.random_normal([10,1]))}
    
    w_h1 = tf.summary.histogram("weights1",weights['hiddenL1'])
    w_h2 = tf.summary.histogram("weights2",weights['hiddenL2'])
    b_h1 = tf.summary.histogram("biases1",biases['hiddenL1'])
    b_h2 = tf.summary.histogram("biases2",biases['hiddenL2'])
    
    
    with tf.name_scope("a1") as scope:
        z1 = tf.matmul(weights['hiddenL1'],x) + biases['hiddenL1']
        a1 = tf.nn.relu(z1)
    
    with tf.name_scope("a2") as scope:
        z2 = tf.matmul(weights['hiddenL2'],a1) + biases['hiddenL2']
    
    regLoss = tf.nn.l2_loss(weights['hiddenL1']) + tf.nn.l2_loss(weights['hiddenL1'])
    return z2, regLoss
    

In [9]:
def logistiticRegression(x):
    W = tf.Variable(tf.random_normal([10,784]))
    b = tf.Variable(tf.zeros([10,1]))
    
    z = tf.matmul(W,x) + b
    regLoss = tf.nn.l2_loss(W)
    return z, regLoss

In [10]:
def convolutionalNeuralNetwork(x):
    weights = {'conv1':tf.Variable(tf.random_normal([5,5,1,32])),
               'conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'fullyC1':tf.Variable(tf.random_normal([7*7*64,1024])),
               'fullyC2':tf.Variable(tf.random_normal([1024,1024])),
               'out':tf.Variable(tf.random_normal([1024,nClasses]))}
    
    biases = {'conv1':tf.Variable(tf.random_normal([32])),
              'conv2':tf.Variable(tf.random_normal([64])),
              'fullyC1':tf.Variable(tf.random_normal([1024])),
              'fullyC2':tf.Variable(tf.random_normal([1024])),
              'out':tf.Variable(tf.random_normal([nClasses]))}
    
    x = tf.transpose(x)
    
    Img = tf.reshape(x,[-1,28,28,1])
    
    conv1 = tf.nn.relu(convolve2D(Img,weights['conv1']) + biases['conv1'])
    
    conv1 = maxpool2d(conv1)
    print(conv1)
    
    conv2 = tf.nn.relu(convolve2D(conv1,weights['conv2']) + biases['conv2'])
#     print(conv2)
    conv2 = maxpool2d(conv2)
    print(conv2)
    
    conv2 = tf.reshape(conv2,[-1,7*7*64])
#     print(conv2)
    fcLayer1 = tf.nn.relu(tf.matmul(conv2,weights['fullyC1']) + biases['fullyC1'])
#     print(fcLayer1)
    fcLayer1 = tf.nn.dropout(fcLayer1,keepRate)
    fcLayer2 = tf.nn.relu(tf.matmul(fcLayer1,weights['fullyC2']) + biases['fullyC2'])
    output = tf.matmul(fcLayer2,weights['out']) + biases['out']
    print(output)
    
    regLoss = tf.nn.l2_loss(weights['conv1']) + tf.nn.l2_loss(weights['conv2']) + tf.nn.l2_loss(weights['fullyC1']) + tf.nn.l2_loss(weights['fullyC1']) + tf.nn.l2_loss(weights['out'])
    return tf.transpose(output), regLoss

In [11]:
def convolutionalNeuralNetwork2(x):
    weights = {'conv1':tf.Variable(tf.random_normal([5,5,1,32])),
               'conv2':tf.Variable(tf.random_normal([5,5,32,32])),
               'conv3':tf.Variable(tf.random_normal([5,5,32,64])),
               'conv4':tf.Variable(tf.random_normal([5,5,64,64])),
               'fullyC1':tf.Variable(tf.random_normal([7*7*64,1024])),
               'fullyC2':tf.Variable(tf.random_normal([1024,1024])),
               'out':tf.Variable(tf.random_normal([1024,nClasses]))}
    
    biases = {'conv1':tf.Variable(tf.random_normal([32])),
              'conv2':tf.Variable(tf.random_normal([32])),
              'conv3':tf.Variable(tf.random_normal([64])),
              'conv4':tf.Variable(tf.random_normal([64])),
              'fullyC1':tf.Variable(tf.random_normal([1024])),
              'fullyC2':tf.Variable(tf.random_normal([1024])),
              'out':tf.Variable(tf.random_normal([nClasses]))}
    
    x = tf.transpose(x)
    
    Img = tf.reshape(x,[-1,28,28,1])
    
    conv1 = tf.nn.relu(convolve2D(Img,weights['conv1']) + biases['conv1'])
    print(conv1)
    conv2 = tf.nn.relu(convolve2D(conv1,weights['conv2']) + biases['conv2'])
    print(conv2)
    conv2 = maxpool2d(conv2)
    print(conv2)
    
    conv3 = tf.nn.relu(convolve2D(conv2,weights['conv3']) + biases['conv3'])
    print(conv3)
    conv4 = tf.nn.relu(convolve2D(conv3,weights['conv4']) + biases['conv4'])
    print(conv4)
    conv4 = maxpool2d(conv4)
    print(conv4)
    
    conv4 = tf.reshape(conv4,[-1,7*7*64])
    print(conv2)
    fcLayer1 = tf.nn.relu(tf.matmul(conv4,weights['fullyC1']) + biases['fullyC1'])
#     print(fcLayer1)
    fcLayer1 = tf.nn.dropout(fcLayer1,keepRate)
    fcLayer2 = tf.nn.relu(tf.matmul(fcLayer1,weights['fullyC2']) + biases['fullyC2'])
    output = tf.matmul(fcLayer2,weights['out']) + biases['out']
    print(output)
    
    return tf.transpose(output)

In [12]:
def trainNetwork():
    with tf.name_scope("loss") as scope:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(y),labels=tf.transpose(t)) + lambdaa*regLoss)
        tf.summary.scalar("loss",loss)
    
    with tf.name_scope("training") as scope:
        optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(loss)
    
    with tf.name_scope("accuracy") as scope:
        correct = tf.equal(tf.argmax(y),tf.argmax(t))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        
    init = tf.global_variables_initializer()
    sess = tf.Session()
#     sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    mergeSummary = tf.summary.merge_all()

    sess.run(init)
    summaryWriter = tf.summary.FileWriter('../TFout/2', sess.graph)

    for epoch in range(nEpochs):
        error = 0.0
        for i in range(int(m/batchSize)):
            batch_xs, batch_ys = mnist.train.next_batch(batchSize)
            _, er, summaryStr = sess.run([optimizer,loss,mergeSummary],feed_dict={x:batch_xs.T,t:batch_ys.T})
            summaryWriter.add_summary(summaryStr, epoch*(int(m/batchSize)) + i)
            error += er
        if (epoch+1)%printEvery == 0:
            print('Loss in ',epoch+1,' epoch is ',error)

    prediction = tf.equal(tf.argmax(y),tf.argmax(t))
    accuracy = tf.reduce_mean(tf.cast(prediction,"float"))
    print("Accuracy Train:", sess.run(accuracy,{x: trainImages, t: trainLabels}))
    print("Accuracy validation:", sess.run(accuracy,{x: validationImages, t: validationLabels}))
    print("Accuracy Test:", sess.run(accuracy,{x: testImages, t: testLabels}))
    print("USPS Test Accuracy:", sess.run(accuracy,{x: uspsImages, t: uspsLabels}))

In [13]:
start_time = time.time()
y, regLoss = logistiticRegression(x)
nEpochs = 50
trainNetwork()
print("--- %s seconds ---" % (time.time() - start_time))

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


NameError: name 'keepRate' is not defined

In [14]:
start_time = time.time()
y, regLoss = neuralNetwork(x)
nEpochs = 25
trainNetwork()
print("--- %s seconds ---" % (time.time() - start_time))

Loss in  2  epoch is  41182.2809377
Loss in  4  epoch is  664.329209864
Loss in  6  epoch is  540.534236193
Loss in  8  epoch is  448.000844508
Loss in  10  epoch is  401.085380554
Loss in  12  epoch is  387.628350526
Loss in  14  epoch is  368.041676462
Loss in  16  epoch is  341.689788938
Loss in  18  epoch is  339.284942836
Loss in  20  epoch is  329.190883189
Loss in  22  epoch is  318.721606761
Loss in  24  epoch is  307.378630012
Accuracy Train: 0.916327
Accuracy validation: 0.9186
Accuracy Test: 0.9187
USPS Test Accuracy: 0.432822
--- 215.52920246124268 seconds ---


In [14]:

nEpochs = 14
keepRate = 0.8
start_time = time.time()
y, regLoss = convolutionalNeuralNetwork(x)
print(y.shape)
print(t.shape)
trainNetwork()
print("--- %s seconds ---" % (time.time() - start_time))

Tensor("MaxPool_2:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("MaxPool_3:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("add_7:0", shape=(?, 10), dtype=float32)
(10, ?)
(10, ?)
Loss in  2  epoch is  66229997.7422


KeyboardInterrupt: 

In [13]:
# nEpochs = 2
# keepRate = 0.7
# start_time = time.time()
# y = convolutionalNeuralNetwork2(x)
# trainNetwork()
# print("--- %s seconds ---" % (time.time() - start_time))

In [13]:

nEpochs = 14
keepRate = 0.8
start_time = time.time()
y, regLoss = convolutionalNeuralNetwork(x)
trainNetwork()
print("--- %s seconds ---" % (time.time() - start_time))

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("add_4:0", shape=(?, 10), dtype=float32)
Loss in  2  epoch is  164789167.344
Loss in  4  epoch is  142050238.672
Loss in  6  epoch is  125193693.078
Loss in  8  epoch is  108960744.297
Loss in  10  epoch is  92894150.6719
Loss in  12  epoch is  77058803.0
Loss in  14  epoch is  62208632.1094
Accuracy Train: 0.991127
Accuracy validation: 0.9802
Accuracy Test: 0.9782
USPS Test Accuracy: 0.611831
--- 1533.0976750850677 seconds ---


In [15]:
learningRate = 0.005
start_time = time.time()
y, regLoss = logistiticRegression(x)
nEpochs = 100
trainNetwork()
print("--- %s seconds ---" % (time.time() - start_time))

Loss in  2  epoch is  1397.80503225
Loss in  4  epoch is  609.988232017
Loss in  6  epoch is  612.54486227
Loss in  8  epoch is  613.822917163
Loss in  10  epoch is  612.926954031
Loss in  12  epoch is  615.499112189
Loss in  14  epoch is  614.427107632
Loss in  16  epoch is  614.569354713
Loss in  18  epoch is  616.080063343
Loss in  20  epoch is  614.699532568
Loss in  22  epoch is  615.160694838
Loss in  24  epoch is  615.188099205
Loss in  26  epoch is  614.722010255
Loss in  28  epoch is  615.799589157
Loss in  30  epoch is  615.280618608
Loss in  32  epoch is  615.502668619
Loss in  34  epoch is  617.265633821
Loss in  36  epoch is  615.239257574
Loss in  38  epoch is  615.147463083
Loss in  40  epoch is  616.363816142
Loss in  42  epoch is  615.513518214
Loss in  44  epoch is  616.159410715
Loss in  46  epoch is  615.841395557
Loss in  48  epoch is  614.631966531
Loss in  50  epoch is  615.743518829
Loss in  52  epoch is  616.380998909
Loss in  54  epoch is  615.194377124
Loss i